# Wykrywanie Anomalii z Rozkładem Normalnym (Gaussa)
## Wstęp teoretyczny
Wykrywanie anomalii to proces identyfikacji danych, które różnią się od większości obserwacji.
W tym ćwiczeniu wykorzystujemy **rozkład normalny (Gaussa)**, aby zmodelować nasze dane i ocenić, które obserwacje mogą być nietypowe.

**Dlaczego rozkład normalny?**
- Dane często mają rozkład zbliżony do normalnego (np. średnie zużycie, opóźnienie itp.).
- Łatwo można oszacować parametry: średnia (`μ`) i odchylenie standardowe (`σ`).
- Można szybko wyznaczyć prawdopodobieństwo dla każdej obserwacji i porównać je z progiem (`epsilon`).

Obserwacje o **bardzo niskim prawdopodobieństwie** względem modelu uważamy za **anomalię**.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.io import loadmat
from scipy.stats import norm
from sklearn.metrics import f1_score

## Wczytanie danych

In [ ]:
data = loadmat('ex8data1.mat')
X = data['X']
print(X.shape)

Import bibliotek do analizy danych, rysowania wykresów i obliczeń statystycznych.

loadmat() wczytuje dane z pliku MATLAB .mat.

X to macierz cech (dwa wymiary: przepustowość i opóźnienie).

X.shape pokazuje liczbę przykładów i wymiarów.



## Wizualizacja danych wejściowych

In [ ]:
x1 = X[:, 0]
x2 = X[:, 1]
plt.scatter(x1, x2, marker='o', s=10, c='blue', alpha=0.5)
plt.xlabel('throughput (mb/s)')
plt.ylabel('latency (ms)')
plt.show()

Opis:

x1, x2 to odpowiednio przepustowość i opóźnienie.

Rysuje wykres punktowy z danymi.



## Histogramy cech

In [ ]:
plt.hist(x1, alpha=0.5, color='blue')
plt.xlabel('throughput (mb/s)')
plt.ylabel('latency (ms)')
plt.show()

In [ ]:
plt.hist(x2, alpha=0.5, color='blue')
plt.xlabel('throughput (mb/s)')
plt.ylabel('latency (ms)')
plt.show()

Histogramy pokazują rozkład cech x1 i x2.

## Estymacja rozkładu normalnego

In [ ]:
def estimate_gaussian(X):
    mu = np.mean(X, axis=0)
    sigma = np.std(X, axis=0)
    return mu, sigma

mu, sigma = estimate_gaussian(X)
print('mu:', mu)
print('sigma:', sigma)

Funkcja oblicza średnie (mu) i odchylenia standardowe (sigma) dla każdej cechy.

Zakłada rozkład normalny danych.

## Dane walidacyjne (do testowania progów wykrywania anomalii)

In [ ]:
Xval = data['Xval']
yval = data['yval']
print("Xval shape:", Xval.shape)
print("Liczba anomalii:", np.sum(yval == 1.0))

Xval – dane walidacyjne, yval – etykiety (1 = anomalia, 0 = normalne).

Wyświetla kształt i liczbę anomalii.



## Krzywa rozkładu normalnego dla cechy x1

In [ ]:
mu1 = mu[0]
sigma1 = sigma[0]
mu2 = mu[1]
sigma2 = sigma[1]

x1_range = np.linspace(x1.min(), x1.max(), 100)
pdf_x1 = norm.pdf(x1_range, loc=mu1, scale=sigma1)

plt.figure(figsize=(8, 6))
plt.plot(x1_range, pdf_x1, color='orange', label='gaussian dist')
plt.scatter(np.sort(x1), norm.pdf(np.sort(x1), loc=mu1, scale=sigma1), label='data points')
plt.xlabel('throughput (mb/s)')
plt.ylabel('Probability')
plt.grid()
plt.legend()
plt.show()

Tworzy wykres funkcji gęstości prawdopodobieństwa (PDF) dla x1.

Porównuje dane do rozkładu Gaussa.

## Obliczenie prawdopodobieństw dla danych walidacyjnych

In [ ]:
pval = norm.pdf(Xval, loc=mu, scale=sigma)
print("Pval shape:", pval.shape)
print("Pval", pval)

Oblicza prawdopodobieństwa dla Xval na podstawie rozkładów Gaussa.

## Funkcja do wyboru najlepszego progu epsilon

In [ ]:
def select_threshold(pval, yval):
    best_epsilon = 0
    best_f1 = 0
    epsilons = np.linspace(pval.min(), pval.max(), 1000)
    for epsilon in epsilons:
        y_pred = (pval < epsilon).astype(int)
        current_f1 = f1_score(yval, y_pred)
        if current_f1 > best_f1:
            best_f1 = current_f1
            best_epsilon = epsilon
    return best_epsilon, best_f1

Szuka najlepszego progu epsilon, który rozdziela dane normalne od anomalii.

Używa metryki F1 do oceny skuteczności.



## Szukanie najlepszego progu i wykrywanie anomalii

In [ ]:
pval1 = norm.pdf(Xval[:, 0], loc=mu1, scale=sigma1)
pval2 = norm.pdf(Xval[:, 1], loc=mu2, scale=sigma2)

best_epsilon_1, best_f1_1 = select_threshold(pval1, yval.flatten())
best_epsilon_2, best_f1_2 = select_threshold(pval2, yval.flatten())

y_pred1 = (pval1 < best_epsilon_1).astype(int)
y_pred2 = (pval2 < best_epsilon_2).astype(int)

print("Najlepszy próg (1. cecha):", best_epsilon_1)
print("Najlepszy F1-score (1. cecha):", best_f1_1)
print("Najlepszy próg (2. cecha):", best_epsilon_2)
print("Najlepszy F1-score (2. cecha):", best_f1_2)

Osobno analizuje każdą cechę (x1, x2) i wybiera optymalne epsilon.

Generuje predykcje i ocenia jakość klasyfikacji.

## Wizualizacja wykrytych anomalii (cecha x1)

In [ ]:
plt.figure(figsize=(6, 5))
plt.scatter(Xval[y_pred1 == 0, 0], Xval[y_pred1 == 0, 1], color='dodgerblue', s=20, label="data points")
plt.scatter(Xval[y_pred1 == 1, 0], Xval[y_pred1 == 1, 1], color='red', s=40, label="anomalies")
plt.xlabel("throughput (mb/s)")
plt.ylabel("latency (ms)")
plt.title("Anomalie wykryte na podstawie cechy x1")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()